In [8]:
# addprocs(30);
addprocs(7);

In [9]:
require(:POMDPModels)
require(:POMCP)

In [10]:
@everywhere begin
    using POMDPModels
    using POMCP
    using POMDPs
    using POMDPToolbox
    import POMCP.init_V
    import POMDPs: action, updater
end

In [11]:
# N = 100000;
N = 100;
problem = BabyPOMDP(-5, -10);

In [12]:
function est_reward(problem, policy, belief, N; eps=0.01)
    sum = @parallel (+) for i in 1:N
        sim_rng = MersenneTwister(i)
        sim = POMDPToolbox.RolloutSimulator(rng=sim_rng, initial_state=false, eps=eps)
        up = updater(policy)
        POMDPs.simulate(sim, problem, policy, up, initialize_belief(up,belief))
    end
    return sum/N;
end

est_reward (generic function with 1 method)

In [13]:
# Feed when Crying (Expected Reward for this nearly-optimal policy is -17.14)
# Test from earlier this week with 5000 experiments: -16.72
@time est_reward(problem, FeedWhenCrying(), false, N)

-15.061634270536405

This is better than in the crying babies test because epsilon is large and, more importantly, it gets a notcrying observation on the first step every time

In [14]:
# Random
pol_rng = MersenneTwister(7)
@time est_reward(problem, RandomPolicy(problem, rng=pol_rng), BoolDistribution(0.5), N)

2.857863 seconds (286.25 k allocations: 12.402 MB, 2.59% gc time)
  

-31.33487686696136

In [15]:
# POMCP with FWC rollout policy
# test from earlier this week with 5000 experiments: -16.77
rng = MersenneTwister(2)

solver = POMCPSolver(rollout_solver=FeedWhenCrying(),
                    eps=0.01,
                    c=10.0,
                    tree_queries=300, 
                    rng=rng,
                    updater=updater(problem))
                    

policy = solve(solver, problem)

@time est_reward(problem, policy, BoolDistribution(0.0), N)

2.497703 seconds (460.17 k allocations: 19.342 MB, 0.60% gc time)
 14

-14.694925426652963

In [16]:
# POMCP with Random rollout policy
rng = MersenneTwister(2)
rollout_pol_rng = MersenneTwister(2)

solver = POMCPSolver(rollout_solver=RandomPolicy(problem, rng=rollout_pol_rng),
                     eps=0.01,
                     c=10.0,
                     tree_queries=300, 
                     rng=rng,
                     updater=updater(problem))

policy = solve(solver, problem)

@time est_reward(problem, policy, BoolDistribution(0.0), N)

.749427 seconds (181.05 k allocations: 7.967 MB)
 12

-15.943953697101506

.209094 seconds (14.79 k allocations: 1.202 MB)


In [17]:
# Optimal policy for these particular problem parameters:
# if the belief that the baby is hungry is over .28206, then feed (see DMU book)
@everywhere begin
    type OptBabyPolicy <: POMDPs.Policy end
    function action(p::OptBabyPolicy, b::BoolDistribution, a=false)
        a = b.p>0.28206
        return a
    end
    updater(::OptBabyPolicy) = updater(BabyPOMDP(-5,-10))
end
@time est_reward(problem, OptBabyPolicy(), BoolDistribution(0.0), N)

-14.686851194997828

0.620187 seconds (46.70 k allocations: 2.346 MB)
